# Classification Exercise

We'll be working with some California Census Data, we'll be trying to use various features of an individual to predict what class of income they belogn in (>50k or <=50k). 

Here is some information about the data:

<table>
<thead>
<tr>
<th>Column Name</th>
<th>Type</th>
<th>Description</th>
</tr>
</thead>
<tbody>
<tr>
<td>age</td>
<td>Continuous</td>
<td>The age of the individual</td>
</tr>
<tr>
<td>workclass</td>
<td>Categorical</td>
<td>The type of employer the  individual has (government,  military, private, etc.).</td>
</tr>
<tr>
<td>fnlwgt</td>
<td>Continuous</td>
<td>The number of people the census  takers believe that observation  represents (sample weight). This  variable will not be used.</td>
</tr>
<tr>
<td>education</td>
<td>Categorical</td>
<td>The highest level of education  achieved for that individual.</td>
</tr>
<tr>
<td>education_num</td>
<td>Continuous</td>
<td>The highest level of education in  numerical form.</td>
</tr>
<tr>
<td>marital_status</td>
<td>Categorical</td>
<td>Marital status of the individual.</td>
</tr>
<tr>
<td>occupation</td>
<td>Categorical</td>
<td>The occupation of the individual.</td>
</tr>
<tr>
<td>relationship</td>
<td>Categorical</td>
<td>Wife, Own-child, Husband,  Not-in-family, Other-relative,  Unmarried.</td>
</tr>
<tr>
<td>race</td>
<td>Categorical</td>
<td>White, Asian-Pac-Islander,  Amer-Indian-Eskimo, Other, Black.</td>
</tr>
<tr>
<td>gender</td>
<td>Categorical</td>
<td>Female, Male.</td>
</tr>
<tr>
<td>capital_gain</td>
<td>Continuous</td>
<td>Capital gains recorded.</td>
</tr>
<tr>
<td>capital_loss</td>
<td>Continuous</td>
<td>Capital Losses recorded.</td>
</tr>
<tr>
<td>hours_per_week</td>
<td>Continuous</td>
<td>Hours worked per week.</td>
</tr>
<tr>
<td>native_country</td>
<td>Categorical</td>
<td>Country of origin of the  individual.</td>
</tr>
<tr>
<td>income</td>
<td>Categorical</td>
<td>"&gt;50K" or "&lt;=50K", meaning  whether the person makes more  than \$50,000 annually.</td>
</tr>
</tbody>
</table>

## Follow the Directions in Bold. If you get stuck, check out the solutions lecture.

### THE DATA

** Read in the census_data.csv data with pandas**

In [1]:
import pandas as pd

In [2]:
census = pd.read_csv("census_data.csv")

In [3]:
census.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


** TensorFlow won't be able to understand strings as labels, you'll need to use pandas .apply() method to apply a custom function that converts them to 0s and 1s. This might be hard if you aren't very familiar with pandas, so feel free to take a peek at the solutions for this part.**

** Convert the Label column to 0s and 1s instead of strings.**

In [4]:
census["income_bracket"].unique()

array([' <=50K', ' >50K'], dtype=object)

In [5]:
def label_fix(label):
    if label == ' <=50K':
        return 0
    else:
        return 1

In [6]:
census["income_bracket"] = census["income_bracket"].apply(label_fix)

In [7]:
census["income_bracket"].unique()

array([0, 1])

### Perform a Train Test Split on the Data

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
x_data = census.drop("income_bracket", axis=1)
y = census["income_bracket"]

X_train, X_test, y_train, y_test = train_test_split(x_data, y, test_size=0.3, random_state=101)

### Create the Feature Columns for tf.esitmator

** Take note of categorical vs continuous values! **

In [10]:
census.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income_bracket'],
      dtype='object')

** Import Tensorflow **

In [11]:
import tensorflow as tf

** Create the tf.feature_columns for the categorical values. Use vocabulary lists or just use hash buckets. **

In [12]:
gender = tf.feature_column.categorical_column_with_vocabulary_list("gender", ["Female", "Male"])
occupation = tf.feature_column.categorical_column_with_hash_bucket("occupation", hash_bucket_size=1000)
marital_status = tf.feature_column.categorical_column_with_hash_bucket("marital_status", hash_bucket_size=1000)
relationship = tf.feature_column.categorical_column_with_hash_bucket("relationship", hash_bucket_size=1000)
education = tf.feature_column.categorical_column_with_hash_bucket("education", hash_bucket_size=1000)
workclass = tf.feature_column.categorical_column_with_hash_bucket("workclass", hash_bucket_size=1000)
native_country = tf.feature_column.categorical_column_with_hash_bucket("native_country", hash_bucket_size=1000)

** Create the continuous feature_columns for the continuous values using numeric_column **

In [13]:
age = tf.feature_column.numeric_column("age")
education_num = tf.feature_column.numeric_column("education_num")
capital_gain = tf.feature_column.numeric_column("capital_gain")
capital_loss = tf.feature_column.numeric_column("capital_loss")
hours_per_week = tf.feature_column.numeric_column("hours_per_week")

** Put all these variables into a single list with the variable name feat_cols **

In [14]:
feat_cols = [gender, occupation, marital_status, relationship, education, workclass, native_country,
             age, education_num, capital_gain, capital_loss, hours_per_week]

### Create Input Function

** Batch_size is up to you. But do make sure to shuffle!**

In [15]:
input_func=tf.estimator.inputs.pandas_input_fn(x=X_train, y=y_train, batch_size=100, num_epochs=None, shuffle=True)

#### Create your model with tf.estimator

**Create a LinearClassifier.(If you want to use a DNNClassifier, keep in mind you'll need to create embedded columns out of the cateogrical feature that use strings, check out the previous lecture on this for more info.)**

In [16]:
model = tf.estimator.LinearClassifier(feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/jf/g8tp5ct56hl6p6r66zl2knzc0000gn/T/tmpy9tza4sp', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a15b41b00>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


** Train your model on the data, for at least 5000 steps. **

In [17]:
model.train(input_fn=input_func,steps=50000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/jf/g8tp5ct56hl6p6r66zl2knzc0000gn/T/tmpy9tza4sp/model.ckpt.
INFO:tensorflow:loss = 69.31472, step = 1
INFO:tensorflow:global_step/sec: 166.805
INFO:tensorflow:loss = 762.61816, step = 101 (0.600 sec)
INFO:tensorflow:global_step/sec: 363.142
INFO:tensorflow:loss = 67.44187, step = 201 (0.277 sec)
INFO:tensorflow:global_step/sec: 361.896
INFO:tensorflow:loss = 148.49289, step = 301 (0.276 sec)
INFO:tensorflow:global_step/sec: 386.415
INFO:tensorflow:loss = 49.76673, step = 401 (0.259 sec)
INFO:tensorflow:global_step/sec: 407.274
INFO:tensorflow:loss = 51.114147, step = 501 (0.246 sec)
INFO:tensorflow:global_step/sec: 395.59
INFO:tensorflow:loss = 22.11246, step = 601 (0.251 sec)
INFO:tensorflow:gl

INFO:tensorflow:loss = 45.509354, step = 8001 (0.270 sec)
INFO:tensorflow:global_step/sec: 305.941
INFO:tensorflow:loss = 45.654118, step = 8101 (0.330 sec)
INFO:tensorflow:global_step/sec: 317.753
INFO:tensorflow:loss = 64.04458, step = 8201 (0.314 sec)
INFO:tensorflow:global_step/sec: 305.623
INFO:tensorflow:loss = 22.617006, step = 8301 (0.327 sec)
INFO:tensorflow:global_step/sec: 301.631
INFO:tensorflow:loss = 34.13812, step = 8401 (0.330 sec)
INFO:tensorflow:global_step/sec: 370.459
INFO:tensorflow:loss = 31.194445, step = 8501 (0.271 sec)
INFO:tensorflow:global_step/sec: 406.152
INFO:tensorflow:loss = 103.617165, step = 8601 (0.246 sec)
INFO:tensorflow:global_step/sec: 394.49
INFO:tensorflow:loss = 47.92203, step = 8701 (0.255 sec)
INFO:tensorflow:global_step/sec: 289.151
INFO:tensorflow:loss = 214.10649, step = 8801 (0.344 sec)
INFO:tensorflow:global_step/sec: 327.115
INFO:tensorflow:loss = 40.30548, step = 8901 (0.308 sec)
INFO:tensorflow:global_step/sec: 296.737
INFO:tensorflo

INFO:tensorflow:global_step/sec: 373.045
INFO:tensorflow:loss = 39.05465, step = 16301 (0.267 sec)
INFO:tensorflow:global_step/sec: 401.731
INFO:tensorflow:loss = 59.386627, step = 16401 (0.249 sec)
INFO:tensorflow:global_step/sec: 346.423
INFO:tensorflow:loss = 47.280872, step = 16501 (0.292 sec)
INFO:tensorflow:global_step/sec: 277.823
INFO:tensorflow:loss = 41.227978, step = 16601 (0.357 sec)
INFO:tensorflow:global_step/sec: 302.745
INFO:tensorflow:loss = 25.367277, step = 16701 (0.330 sec)
INFO:tensorflow:global_step/sec: 321.632
INFO:tensorflow:loss = 28.16309, step = 16801 (0.309 sec)
INFO:tensorflow:global_step/sec: 326.133
INFO:tensorflow:loss = 55.34558, step = 16901 (0.307 sec)
INFO:tensorflow:global_step/sec: 368.23
INFO:tensorflow:loss = 56.587433, step = 17001 (0.273 sec)
INFO:tensorflow:global_step/sec: 387.863
INFO:tensorflow:loss = 27.098656, step = 17101 (0.256 sec)
INFO:tensorflow:global_step/sec: 382
INFO:tensorflow:loss = 37.906136, step = 17201 (0.263 sec)
INFO:ten

INFO:tensorflow:loss = 34.130287, step = 24501 (0.258 sec)
INFO:tensorflow:global_step/sec: 397.122
INFO:tensorflow:loss = 96.462494, step = 24601 (0.252 sec)
INFO:tensorflow:global_step/sec: 377.97
INFO:tensorflow:loss = 101.48118, step = 24701 (0.266 sec)
INFO:tensorflow:global_step/sec: 369.827
INFO:tensorflow:loss = 32.51805, step = 24801 (0.271 sec)
INFO:tensorflow:global_step/sec: 394.643
INFO:tensorflow:loss = 41.923534, step = 24901 (0.253 sec)
INFO:tensorflow:global_step/sec: 406.449
INFO:tensorflow:loss = 37.876404, step = 25001 (0.245 sec)
INFO:tensorflow:global_step/sec: 283.74
INFO:tensorflow:loss = 36.979824, step = 25101 (0.352 sec)
INFO:tensorflow:global_step/sec: 305.403
INFO:tensorflow:loss = 31.613787, step = 25201 (0.327 sec)
INFO:tensorflow:global_step/sec: 317.329
INFO:tensorflow:loss = 36.334408, step = 25301 (0.317 sec)
INFO:tensorflow:global_step/sec: 330.775
INFO:tensorflow:loss = 97.75553, step = 25401 (0.300 sec)
INFO:tensorflow:global_step/sec: 393.185
INFO

INFO:tensorflow:global_step/sec: 344.521
INFO:tensorflow:loss = 34.31942, step = 32801 (0.290 sec)
INFO:tensorflow:global_step/sec: 300.752
INFO:tensorflow:loss = 62.23371, step = 32901 (0.332 sec)
INFO:tensorflow:global_step/sec: 331.053
INFO:tensorflow:loss = 33.767475, step = 33001 (0.303 sec)
INFO:tensorflow:global_step/sec: 345.221
INFO:tensorflow:loss = 31.033953, step = 33101 (0.289 sec)
INFO:tensorflow:global_step/sec: 369.88
INFO:tensorflow:loss = 38.087906, step = 33201 (0.273 sec)
INFO:tensorflow:global_step/sec: 371.876
INFO:tensorflow:loss = 23.234318, step = 33301 (0.265 sec)
INFO:tensorflow:global_step/sec: 370.987
INFO:tensorflow:loss = 31.07195, step = 33401 (0.272 sec)
INFO:tensorflow:global_step/sec: 382.358
INFO:tensorflow:loss = 28.63946, step = 33501 (0.260 sec)
INFO:tensorflow:global_step/sec: 396.825
INFO:tensorflow:loss = 111.97334, step = 33601 (0.252 sec)
INFO:tensorflow:global_step/sec: 403.736
INFO:tensorflow:loss = 34.13626, step = 33701 (0.249 sec)
INFO:t

INFO:tensorflow:loss = 40.394127, step = 41001 (0.269 sec)
INFO:tensorflow:global_step/sec: 355.145
INFO:tensorflow:loss = 28.810232, step = 41101 (0.285 sec)
INFO:tensorflow:global_step/sec: 363.775
INFO:tensorflow:loss = 32.975376, step = 41201 (0.271 sec)
INFO:tensorflow:global_step/sec: 276.216
INFO:tensorflow:loss = 32.99862, step = 41301 (0.362 sec)
INFO:tensorflow:global_step/sec: 350.776
INFO:tensorflow:loss = 31.622782, step = 41401 (0.286 sec)
INFO:tensorflow:global_step/sec: 386.475
INFO:tensorflow:loss = 33.92857, step = 41501 (0.259 sec)
INFO:tensorflow:global_step/sec: 370.914
INFO:tensorflow:loss = 33.349598, step = 41601 (0.272 sec)
INFO:tensorflow:global_step/sec: 292.983
INFO:tensorflow:loss = 31.091478, step = 41701 (0.340 sec)
INFO:tensorflow:global_step/sec: 336.066
INFO:tensorflow:loss = 33.048073, step = 41801 (0.298 sec)
INFO:tensorflow:global_step/sec: 374.267
INFO:tensorflow:loss = 39.462036, step = 41901 (0.267 sec)
INFO:tensorflow:global_step/sec: 375.116
IN

INFO:tensorflow:global_step/sec: 348.44
INFO:tensorflow:loss = 31.282978, step = 49301 (0.288 sec)
INFO:tensorflow:global_step/sec: 407.349
INFO:tensorflow:loss = 31.294193, step = 49401 (0.245 sec)
INFO:tensorflow:global_step/sec: 394.034
INFO:tensorflow:loss = 43.918015, step = 49501 (0.253 sec)
INFO:tensorflow:global_step/sec: 369.988
INFO:tensorflow:loss = 28.918892, step = 49601 (0.270 sec)
INFO:tensorflow:global_step/sec: 394.08
INFO:tensorflow:loss = 35.995384, step = 49701 (0.255 sec)
INFO:tensorflow:global_step/sec: 404.244
INFO:tensorflow:loss = 42.367004, step = 49801 (0.247 sec)
INFO:tensorflow:global_step/sec: 410.553
INFO:tensorflow:loss = 38.713535, step = 49901 (0.242 sec)
INFO:tensorflow:Saving checkpoints for 50000 into /var/folders/jf/g8tp5ct56hl6p6r66zl2knzc0000gn/T/tmpy9tza4sp/model.ckpt.
INFO:tensorflow:Loss for final step: 24.951155.


### Evaluation

** Create a prediction input function. Remember to only supprt X_test data and keep shuffle=False. **

In [18]:
pred_fn = tf.estimator.inputs.pandas_input_fn(x=X_test, batch_size=len(X_test), shuffle=False)

** Use model.predict() and pass in your input function. This will produce a generator of predictions, which you can then transform into a list, with list() **

In [19]:
predictions = list(model.predict(input_fn=pred_fn))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/jf/g8tp5ct56hl6p6r66zl2knzc0000gn/T/tmpy9tza4sp/model.ckpt-50000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


** Each item in your list will look like this: **

In [20]:
predictions[0]

{'logits': array([-0.74437225], dtype=float32),
 'logistic': array([0.3220488], dtype=float32),
 'probabilities': array([0.6779512, 0.3220488], dtype=float32),
 'class_ids': array([0]),
 'classes': array([b'0'], dtype=object)}

** Create a list of only the class_ids key values from the prediction list of dictionaries, these are the predictions you will use to compare against the real y_test values. **

In [21]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['class_ids'][0])

In [22]:
final_preds[:10]

[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]

** Import classification_report from sklearn.metrics and then see if you can figure out how to use it to easily get a full report of your model's performance on the test data. **

In [23]:
from sklearn.metrics import classification_report

In [24]:
print(classification_report(y_test,final_preds))

             precision    recall  f1-score   support

          0       0.89      0.92      0.90      7436
          1       0.71      0.63      0.67      2333

avg / total       0.84      0.85      0.85      9769

